In [ ]:
import random
import numpy as np
import random,os
import torch
import pandas as pd
from pathlib import Path
import librosa
from matplotlib import pyplot as plt
import IPython

from PIL import Image
import ipywidgets as widgets
from ipywidgets import Layout
from matplotlib import pyplot as plt

from glob import glob
from IPython.display import display

#Deep learning from pytorch
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

!pip install timm
import timm

In [ ]:
import json
json_open = open('/kaggle/input/birdclef-2022/scored_birds.json', 'r')
bird_target = json.load(json_open)
bird_target

In [ ]:
class Model(nn.Module):
    def __init__(self,name,pretrained=False):
        super(Model, self).__init__()
        self.model = timm.create_model(name,pretrained=pretrained, in_chans=1)
        self.model.reset_classifier(num_classes=0) 
        in_features = self.model.num_features
        self.fc = nn.Linear(in_features, 21)

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x
    
paths = [
    "../input/bridclef2022dataset/eca_nfnet_l0_p7CV0.881.bin",
    "../input/bridclef2022dataset/resnest50d_CV0.852.bin",
    "../input/bridclef2022dataset/convnext_tiny_CV0.83.bin",
]

names = [
    "eca_nfnet_l0",
    "resnest50d",
    "convnext_tiny",
]
    
models = []
for path, name in zip(paths, names):
    model = Model(name=name)#.to(device)
    model.load_state_dict(torch.load(path,map_location=torch.device('cpu')))
    model.eval()
    models.append(model)

In [ ]:
train = pd.read_csv("../input/fulllabeldataset/train.csv",index_col=0)
train = train[train.num != 0]
train = train[train.primary_label.isin(bird_target)]
train["filepath"] = "../input/fulllabeldataset/image_128/" + train["path"]
train["sort_index"] = train.filename_id.apply(lambda x: int(x.split("_")[-1])//5)
train["file_id"] = train.filename_id.apply(lambda x: x.split("_")[0])
train['secondary_labels_array'] =  train.secondary_labels.apply(eval)
train["sleng"] = train.secondary_labels_array.apply(lambda x: len(x))
unique_key = sorted(bird_target)
LABEL_IDS = {label: label_id for label_id, label in enumerate(unique_key)}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}
print(LABEL_IDS)
train["label_id"] = train["primary_label"].map(LABEL_IDS)

df_label = train[train.sleng > 0]
df_label['labels_id'] = df_label.secondary_labels_array.apply(lambda x: np.vectorize(lambda s: LABEL_IDS[s] if s in unique_key else -1)(x))
df_label.labels_id = df_label.labels_id.apply(lambda x: x[x != -1])
df_label.labels_id = df_label.apply(lambda x: x.labels_id[x.labels_id != x.label_id],axis=1)
df_label['num'] = df_label.labels_id.apply(lambda x: len(x))
df_label = df_label[df_label.num != 0]
df_label["sec1"] = df_label.labels_id.apply(lambda x: x[0])
df_label["sec2"] = df_label.labels_id.apply(lambda x: x[1] if len(x) > 1 else 0)
df_label["sec3"] = df_label.labels_id.apply(lambda x: x[2] if len(x) > 2 else 0)
df_label["sec4"] = df_label.labels_id.apply(lambda x: x[3] if len(x) > 3 else 0)
df_label["sec5"] = df_label.labels_id.apply(lambda x: x[4] if len(x) > 4 else 0)
df_label["sec6"] = df_label.labels_id.apply(lambda x: x[5] if len(x) > 5 else 0)



df_label["label_array"] = 0
df_label.loc[df_label.num == 0, "label_array"] = df_label.label_id.apply(lambda x: np.array([x]))
df_label.loc[df_label.num != 0, "label_array"] = df_label.loc[df_label.num != 0 ,["labels_id","label_id"]].apply(lambda x: np.append(x.labels_id,x.label_id),axis=1)
train = pd.concat([df_label]).sort_values(["file_id","sort_index"]).reset_index(drop=True)

pathdf = pd.DataFrame(glob("../input/birdclef-2022/train_audio/**/*.ogg"),columns=["audio_path"])
pathdf["file_id"] = pathdf.audio_path.apply(lambda x: x.split("/")[-1].replace(".ogg",""))
train = pd.merge(train,pathdf,on=["file_id"])
train

In [ ]:
#reflect previous results
!cp ../input/handlabelingnotebook/results.txt /kaggle/working/results.txt

In [ ]:
# train = train[(train.num == 3)]#&(train.sleng >= 2)
# train = train[~train.primary_label.isin(["houfin","skylar"])]
print(len(train))


#train = train[~train.primary_label.isin(["skylar","houfin",])]

In [ ]:
train.primary_label.value_counts()

In [ ]:
class AnnotationOnJupyter:
    def __init__(self, df, labels=np.append(bird_target,"nocall")) -> None:

        self.df = df
        self.labels = labels
        self.prev_img_id = ""

        self.output = widgets.Output()
        self.ax = plt.gca()
        
        self.iter = 0
        self.display = None

        btns = []
        for label in labels:
            btn = widgets.Button(description=label)
            btn.on_click(self.on_click)
            btns.append(btn)
        self.btns = btns

    def start(self):
        
        box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='500pt'
        )
        box_layout2 = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='700pt'
        )
        box1 = widgets.Box(self.btns[0:5], layout=box_layout)
        box2 = widgets.Box(self.btns[5:10], layout=box_layout)
        box3 = widgets.Box(self.btns[10:15], layout=box_layout)
        box4 = widgets.Box(self.btns[15:], layout=box_layout2)
        box = widgets.VBox([box1, box2, box3, box4])
        
        #display(*self.btns, self.output)
        display(box, self.output)
        
        plt.close()

        blank_btn = widgets.Button(description="blank")
        blank_btn.on_click(lambda x: self.on_click(x))
        blank_btn.click()

    def on_click(self, b: widgets.Button) -> None:
        row = self.df.iloc[self.iter]
        if self.prev_img_id != "":
            with open("./results_sec3_3.txt", "a") as f:
                f.write(f"{self.prev_img_id},{b.description}\n")

        self.ax.clear()
        
        img_path = row.filepath
        self.prev_img_id = row.filename_id
        img = np.load(img_path)
        image = torch.tensor(img.astype("float32", copy=False) / 255.0)[None,None,:,:]
        for idx,model in enumerate(models):
            pred = model(image)
            if idx==0:
                preds = pred.sigmoid().detach().cpu().numpy()
            else:
                preds += pred.sigmoid().detach().cpu().numpy()
            
        preds = preds/len(models)
        preds = np.array([round(pred,2) for pred in preds[0]])
        self.ax.set_title(f"{self.prev_img_id}_{row.primary_label}_targetlabel:{bird_target[row.sec1]}_seclabel:{row.secondary_labels}_secprob:{preds[row.label_array]}")
        self.ax.imshow(img)
        
        
        y, sr = librosa.load(row.audio_path, offset=int(row["sort_index"]*5), duration=20)
        

        with self.output:
            self.output.clear_output(wait=True)
            display(self.ax.figure)
            display(IPython.display.Audio(data=y, rate=sr))
        
            
        self.iter += 1

In [ ]:
if __name__ == "__main__":
    annotator = AnnotationOnJupyter(train)
    annotator.start()